In [2]:
# importing packages

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Depois de uns meses a fazer candidatures e falares com pessoas no LinkedIn para conseguires um um estágio de Verão, finalmente és aceite pela empresa **LabOfLife (LOL)**

Estamos a 1 de Junho, no primeiro dia do teu estágio. Acabaste de entraste numa nova equipa de laboratório que analisa proteínas e as suas propriedades. Nos primeiros dias és introduzido aos diversos projetos que o laboratório tem e também à equipa onde te vais inserir, e mal podes esperar por começar a aprender.

Estamos no 3º dia, chegas cedo ao laboratório e são 9 da manhã. Mal entras na sala da tua equipa, ficas surpreso por ver toda a gente já lá e ainda por cima reparas nas olheiras assutadoras que todos têm. Preocupado, chegas-te a uma colega e perguntas o que se está a passar. Ela, cansada, diz-te: 


> "Estamos a analisar as fosforilações presentes num proteoma de alfa-casína. Para isso fizémos uma experiência e recolhemos os dados todos para o ficheiro `data/dados.csv`. No entanto passámos a noite toda a tentar terminar a nossa análise das fosforilações, mas a única coisa que conseguimos fazer foi uma função que está no ficheiro `utils.py` que extrai só os eventos de fosforilações da coluna que contém todas as modificações que aconteceram às proteínas"

Aprontamente, tu ofereces a tua ajuda:

> "Não te preocupes. Eu fiz um workshop de Pandas e Python organizado pelo ANEEB e consigo-vos ajudar, explica-me melhor a situação e eu resolvo isto em menos de 1h."

O resto da sala procede a explicar-te exatamente o problema, de seguida agradecem-te e foram descansar para casa.


**Contexto**:
> As fosforilações são modificações que ocorrem nos aminoácidos S, T e Y. São eventos dinâmicos e de baixa expressão. No proteoma total estes têm que ser pré-concentrados antes da análise por espectrometria de massa.
Para isso existem inúmeras opções, mas nem todas as proteínas têm a mesma afinidade para o tipo de sequência, aminoácido fosforilado ou até mesmo para a multiplicidade de fosforilações numa determinada sequência. 

**Problema**:

> Então, aqui vamos querer analisar o número total de fosforilações por sequência, bem como o número de fosforilações que ocorrem nos aminoácidos S, T e Y.

# Exercise 1 - Read the data

In [26]:
df=pd.read_csv('data/dados_fosforilacoes_aminoacidos.csv')

#df.head() you may uncomment this to inspect the data

Evaluation cell below

In [14]:
# don't forget to hide contents by default
assert df.shape==(2541,20), "Oops!"

# Exercise 2 - Funnelling down

There's a lot of information on this dataset, but you remember that the problem consists of analyzing only the phosphorylation events out of all **Modification** that occurred on the **sequence of aminoacids**. Look for the 2 columns that contain this information and retrieve a new dataframe consisting only of these columns

In [22]:
protein_reactions_df = df[['Sequence','Modifications']]

In [23]:
protein_reactions_df.head(3)
protein_reactions_df.shape

,Sequence,Modifications
0,K.TVDMESTEVFTK.K,Acetyl: 1; Oxidation: 4; Phospho: 6
1,K.VPQLEIVPNSAEER.L,Phospho: 10
2,K.VPQLEIVPNSAEER.L,Phospho: 10


(2541, 2)

In [24]:
assert protein_reactions_df.shape==(2541,2), "Oops!"